### Execution Plan: explain()


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('app1').getOrCreate()
df = spark.read.csv("customer.csv",header=True)

In [7]:
df.select('Customer_No')

DataFrame[Customer_No: string]

In [8]:
df.select('Customer_No').explain()

== Physical Plan ==
FileScan csv [Customer_No#16] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/nares/PySpark_Examples/customer.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<Customer_No:string>




In [9]:
df.show()

+-----------+-------------+---------+-------------------+----------------+-------+
|Customer_No|    Card_type|     Date|           Category|Transaction Type| Amount|
+-----------+-------------+---------+-------------------+----------------+-------+
|    1000501|Platinum Card| 1/1/2018|           Shopping|           debit|  11.11|
|    1000501|     Checking| 1/2/2018|    Mortgage & Rent|           debit|1247.44|
|    1000501|  Silver Card| 1/2/2018|        Restaurants|           debit|  24.22|
|    1000501|Platinum Card| 1/3/2018|Credit Card Payment|          credit|2298.09|
|    1000501|Platinum Card| 1/4/2018|      Movies & DVDs|           debit|  11.76|
|    1000501|  Silver Card| 1/5/2018|        Restaurants|           debit|  25.85|
|    1000501|  Silver Card| 1/6/2018|   Home Improvement|           debit|  18.45|
|    1000501|     Checking| 1/8/2018|          Utilities|           debit|     45|
|    1000501|  Silver Card| 1/8/2018|   Home Improvement|           debit|  15.38|
|   

In [11]:
from pyspark.sql.functions import col

df.filter("Amount >= 10").groupBy("Card_Type").count().show()

+-------------+-----+
|    Card_Type|count|
+-------------+-----+
|  Silver Card|   28|
|Platinum Card|   35|
|     Checking|   32|
+-------------+-----+



In [13]:
df.filter("Amount >= 10").groupBy("Card_Type").count().explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Card_Type#17], functions=[count(1)])
   +- Exchange hashpartitioning(Card_Type#17, 200), ENSURE_REQUIREMENTS, [plan_id=241]
      +- HashAggregate(keys=[Card_Type#17], functions=[partial_count(1)])
         +- Project [Card_type#17]
            +- Filter (isnotnull(Amount#21) AND (cast(Amount#21 as int) >= 10))
               +- FileScan csv [Card_type#17,Amount#21] Batched: false, DataFilters: [isnotnull(Amount#21), (cast(Amount#21 as int) >= 10)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/nares/PySpark_Examples/customer.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Amount)], ReadSchema: struct<Card_type:string,Amount:string>




In [14]:
df.filter("Amount >= 10").groupBy("Card_Type").count().explain("simple")

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Card_Type#17], functions=[count(1)])
   +- Exchange hashpartitioning(Card_Type#17, 200), ENSURE_REQUIREMENTS, [plan_id=258]
      +- HashAggregate(keys=[Card_Type#17], functions=[partial_count(1)])
         +- Project [Card_type#17]
            +- Filter (isnotnull(Amount#21) AND (cast(Amount#21 as int) >= 10))
               +- FileScan csv [Card_type#17,Amount#21] Batched: false, DataFilters: [isnotnull(Amount#21), (cast(Amount#21 as int) >= 10)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/nares/PySpark_Examples/customer.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Amount)], ReadSchema: struct<Card_type:string,Amount:string>




In [15]:
df.filter("Amount >= 10").groupBy("Card_Type").count().explain("extended")

== Parsed Logical Plan ==
'Aggregate ['Card_Type], ['Card_Type, count(1) AS count#256L]
+- Filter (cast(Amount#21 as int) >= 10)
   +- Relation [Customer_No#16,Card_type#17,Date#18,Category#19,Transaction Type#20,Amount#21] csv

== Analyzed Logical Plan ==
Card_Type: string, count: bigint
Aggregate [Card_Type#17], [Card_Type#17, count(1) AS count#256L]
+- Filter (cast(Amount#21 as int) >= 10)
   +- Relation [Customer_No#16,Card_type#17,Date#18,Category#19,Transaction Type#20,Amount#21] csv

== Optimized Logical Plan ==
Aggregate [Card_Type#17], [Card_Type#17, count(1) AS count#256L]
+- Project [Card_type#17]
   +- Filter (isnotnull(Amount#21) AND (cast(Amount#21 as int) >= 10))
      +- Relation [Customer_No#16,Card_type#17,Date#18,Category#19,Transaction Type#20,Amount#21] csv

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Card_Type#17], functions=[count(1)], output=[Card_Type#17, count#256L])
   +- Exchange hashpartitioning(Card_Type#17, 200), ENSURE_

In [16]:
df.filter("Amount >= 10").groupBy("Card_Type").count().explain("cost")

== Optimized Logical Plan ==
Aggregate [Card_Type#17], [Card_Type#17, count(1) AS count#268L], Statistics(sizeInBytes=1596.0 B)
+- Project [Card_type#17], Statistics(sizeInBytes=1242.0 B)
   +- Filter (isnotnull(Amount#21) AND (cast(Amount#21 as int) >= 10)), Statistics(sizeInBytes=5.5 KiB)
      +- Relation [Customer_No#16,Card_type#17,Date#18,Category#19,Transaction Type#20,Amount#21] csv, Statistics(sizeInBytes=5.5 KiB)

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[Card_Type#17], functions=[count(1)], output=[Card_Type#17, count#268L])
   +- Exchange hashpartitioning(Card_Type#17, 200), ENSURE_REQUIREMENTS, [plan_id=292]
      +- HashAggregate(keys=[Card_Type#17], functions=[partial_count(1)], output=[Card_Type#17, count#272L])
         +- Project [Card_type#17]
            +- Filter (isnotnull(Amount#21) AND (cast(Amount#21 as int) >= 10))
               +- FileScan csv [Card_type#17,Amount#21] Batched: false, DataFilters: [isnotnull(Amount#21), (c

In [25]:
spark.conf.set("spark.sql.adaptive.enabled",False)
df.filter("Amount >= 10").groupBy("Card_Type").count().explain("codegen")

Found 2 WholeStageCodegen subtrees.
== Subtree 1 / 2 (maxMethodCodeSize:283; maxConstantPoolSize:269(0.41% used); numInnerClasses:2) ==
*(1) HashAggregate(keys=[Card_Type#17], functions=[partial_count(1)], output=[Card_Type#17, count#354L])
+- *(1) Project [Card_type#17]
   +- *(1) Filter (isnotnull(Amount#21) AND (cast(Amount#21 as int) >= 10))
      +- FileScan csv [Card_type#17,Amount#21] Batched: false, DataFilters: [isnotnull(Amount#21), (cast(Amount#21 as int) >= 10)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/C:/Users/nares/PySpark_Examples/customer.csv], PartitionFilters: [], PushedFilters: [IsNotNull(Amount)], ReadSchema: struct<Card_type:string,Amount:string>

Generated code:
/* 001 */ public Object generate(Object[] references) {
/* 002 */   return new GeneratedIteratorForCodegenStage1(references);
/* 003 */ }
/* 004 */
/* 005 */ // codegenStageId=1
/* 006 */ final class GeneratedIteratorForCodegenStage1 extends org.apache.spark.sql.execution.BufferedRowIterato

In [26]:
df.filter("Amount >= 10").groupBy("Card_Type").count().explain("formatted")

== Physical Plan ==
* HashAggregate (6)
+- Exchange (5)
   +- * HashAggregate (4)
      +- * Project (3)
         +- * Filter (2)
            +- Scan csv  (1)


(1) Scan csv 
Output [2]: [Card_type#17, Amount#21]
Batched: false
Location: InMemoryFileIndex [file:/C:/Users/nares/PySpark_Examples/customer.csv]
PushedFilters: [IsNotNull(Amount)]
ReadSchema: struct<Card_type:string,Amount:string>

(2) Filter [codegen id : 1]
Input [2]: [Card_type#17, Amount#21]
Condition : (isnotnull(Amount#21) AND (cast(Amount#21 as int) >= 10))

(3) Project [codegen id : 1]
Output [1]: [Card_type#17]
Input [2]: [Card_type#17, Amount#21]

(4) HashAggregate [codegen id : 1]
Input [1]: [Card_type#17]
Keys [1]: [Card_Type#17]
Functions [1]: [partial_count(1)]
Aggregate Attributes [1]: [count#365L]
Results [2]: [Card_Type#17, count#366L]

(5) Exchange
Input [2]: [Card_Type#17, count#366L]
Arguments: hashpartitioning(Card_Type#17, 200), ENSURE_REQUIREMENTS, [plan_id=455]

(6) HashAggregate [codegen id : 2]
Inpu